# 22.3. Сложная параметризация тестов
Итак, возьмем метод API из нашего сервиса — /api/create_pet_simple. Видим, что у него перечислены параметры, которые обязательно должны быть переданы: auth_key, name, animal_type, age. Других ограничений на эти параметры не написано, поэтому мы будем считать, что наш сервис должен возвращать ошибку только тогда, когда мы передаем в один из этих параметров пустую строку. В противном случае мы ожидаем увидеть сущность питомца именно с теми значениями, которые передавали, в ответе на запрос (да, даже с китайскими символами, мы предполагаем, что наш сервис будет международным). Для начала добавим в наш файл api.py, который умеет работать с API сервиса, еще одну функцию create_pet_simple по аналогии с create_pet. Разница будет только в наличии файла, который мы пытаемся запостить:


In [ ]:
def add_new_pet_simple(self, auth_key: json, name: str, animal_type: str, age: str) -> json:
   """Метод отправляет (постит) на сервер данные о добавляемом питомце и возвращает статус
   запроса и результат в формате JSON с данными добавленного питомца"""

   data = MultipartEncoder(
       fields={
           'name': name,
           'animal_type': animal_type,
           'age': age
       })
   headers = {'auth_key': auth_key['key'], 'Content-Type': data.content_type}

   res = requests.post(self.base_url + 'api/create_pet_simple', headers=headers, data=data)
   status = res.status_code
   result = ""
   try:
       result = res.json()
   except json.decoder.JSONDecodeError:
       result = res.text
   print(result)
   return status, result

Теперь возьмем наш тест test_add_new_pet_with_valid_data и создадим по аналогии другой тест. Назовем его test_add_new_pet_simple и параметризуем по принципу, описанному выше. Единственное, что бросается в глаза — это то, что мы пытаемся передать число в виде строки, но это не проблема, так как form-data тип данных не может принимать числовые данные в принципе, у нас просто не остаётся других вариантов:

In [ ]:
@pytest.mark.parametrize("name", [
   ''
   , generate_string(255)
   , generate_string(1001)
   , russian_chars()
   , russian_chars().upper()
   , chinese_chars()
   , special_chars()
   , '123'
], ids=[
   'empty'
   , '255 symbols'
   , 'more than 1000 symbols'
   , 'russian'
   , 'RUSSIAN'
   , 'chinese'
   , 'specials'
   , 'digit'
])
def test_add_new_pet_simple(name, animal_type='двортерьер',
                           age='4'):
   """Проверяем, что можно добавить питомца с различными данными"""

   # Добавляем питомца
   pytest.status, result = pf.add_new_pet_simple(pytest.key, name, animal_type, age)

   # Сверяем полученный ответ с ожидаемым результатом
   if name == '':
       assert pytest.status == 400
   else:
	 assert pytest.status == 200
       assert result['name'] == name
       assert result['age'] == age
       assert result['animal_type'] == animal_type

ак видно из теста, мы проверяем, что код ответа 200 только в том случае, если значение параметра name не пустое.

Теперь добавим параметризацию по еще одному параметру — animal_type. Все, что нам нужно будет сделать, — это добавить в блок проверки условие, что animal_type точно так же не может быть пустой строкой:

In [ ]:
@pytest.mark.parametrize("name"
   , ['', generate_string(255), generate_string(1001), russian_chars(), russian_chars().upper(), chinese_chars(), special_chars(), '123']
   , ids=['empty', '255 symbols', 'more than 1000 symbols', 'russian', 'RUSSIAN', 'chinese', 'specials', 'digit'])
@pytest.mark.parametrize("animal_type"
   , ['', generate_string(255), generate_string(1001), russian_chars(), russian_chars().upper(), chinese_chars(), special_chars(), '123']
   , ids=['empty', '255 symbols', 'more than 1000 symbols', 'russian', 'RUSSIAN', 'chinese', 'specials', 'digit'])
def test_add_new_pet_simple(name, animal_type,
                           age='4'):
   """Проверяем, что можно добавить питомца с различными данными"""

   # Добавляем питомца
   pytest.status, result = pf.add_new_pet_simple(pytest.key, name, animal_type, age)

   # Добавили проверку animal_type
   if name == '' or animal_type == '':
       assert pytest.status == 400
   else:
       assert pytest.status == 200
       assert result['name'] == name
       assert result['age'] == age
       assert result['animal_type'] == animal_type

В несколько движений наших тестов стало в два раза больше — 64 вместо 8. Осталось ещё обработать логику параметра age и точно так же передать его в параметризованный тест. Не забудем подумать о классах эквивалентности для числа.
Итого имеем для параметра age:

-1 # Отрицательного возраста не бывает.
0 # Граничное значение перед минимально возможным возрастом.
1  # Граничное значение (минимально возможный возраст).
100  # Попытка поймать баг логики приложения — мало кто из питомцев живёт до 100 лет, поэтому передаём значение, больше максимально допустимого возраста.
1.5 # Пытаемся передать дробное значение возраста.
2 147 483 647 # Ограничение целочисленного типа данных.
2 147 483 648 # Превышение ограничения целочисленного типа данных.
Спецсимволы # Не принимаем текст в любом виде.
Текст # Не принимаем текст в любом виде.
Различные языки (русский, китайский) # Не принимаем текст в любом виде.

In [ ]:
def is_age_valid(age):
   # Проверяем, что возраст - это число от 1 до 49 и целое
   return age.isdigit() \
          and 0 < int(age) < 50 \
          and float(age) == int(age)


@pytest.mark.parametrize("name"
   , ['', generate_string(255), generate_string(1001), russian_chars(), russian_chars().upper(), chinese_chars(), special_chars(), '123']
   , ids=['empty', '255 symbols', 'more than 1000 symbols', 'russian', 'RUSSIAN', 'chinese', 'specials', 'digit'])
@pytest.mark.parametrize("animal_type"
   , ['', generate_string(255), generate_string(1001), russian_chars(), russian_chars().upper(), chinese_chars(), special_chars(), '123']
   , ids=['empty', '255 symbols', 'more than 1000 symbols', 'russian', 'RUSSIAN', 'chinese', 'specials', 'digit'])
@pytest.mark.parametrize("age"
   , ['', '-1', '0', '1', '100', '1.5', '2147483647', '2147483648', special_chars(), russian_chars(), russian_chars().upper(), chinese_chars()]
   , ids=['empty', 'negative', 'zero', 'min', 'greater than max', 'float', 'int_max', 'int_max + 1', 'specials', 'russian', 'RUSSIAN', 'chinese'])
def test_add_new_pet_simple(name, animal_type, age):
   """Проверяем, что можно добавить питомца с различными данными"""

   # Добавляем питомца
   pytest.status, result = pf.add_new_pet_simple(pytest.key, name, animal_type, age)

   # Сверяем полученный ответ с ожидаемым результатом
   if name == '' or animal_type == '' or is_age_valid(age):
       assert pytest.status == 400
   else:
       assert pytest.status == 200
       assert result['name'] == name
       assert result['age'] == age
       assert result['animal_type'] == animal_type